In [1]:
import logging
import argparse

import onnx
import yaml

from pycocotools.coco import COCO
from pycocotools.mask import iou, encode
import numpy as np
from torchvision import transforms
from PIL import Image
from onnx import numpy_helper
import os
import onnxruntime

In [4]:
# key = COCO id, value = Pascal VOC id
COCO_TO_VOC = {
    1: 15,  # person
    2: 2,   # bicycle
    3: 7,   # car
    4: 14,  # motorbike
    5: 1,   # airplane
    6: 6,   # bus
    7: 19,  # train
    9: 4,   # boat
    16: 3,  # bird
    17: 8,  # cat
    18: 12, # dog
    19: 13, # horse
    20: 17, # sheep
    21: 10, # cow
    44: 5,  # bottle
    62: 9,  # chair
    63: 18, # couch/sofa
    64: 16, # potted plant
    67: 11, # dining table
    72: 20, # tv
}

VOC_CAT_IDS = list(COCO_TO_VOC.keys())
cocoGt = COCO(str("data/annotations/instances_val2017.json"))

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


In [5]:
class Dataset:
    def __init__(self):
        imgIds = self.getImgIdsUnion(cocoGt, VOC_CAT_IDS)
        self.data = []
        for imgId in imgIds:
            img_path = os.path.join("data/val2017_sub/", cocoGt.imgs[imgId]['file_name'])
            if os.path.exists(img_path):
                input_tensor = self.load_image(img_path)
                
                _, height, width = input_tensor.shape
                output_tensor = np.zeros((21, height, width), dtype=np.uint8)
                
                annIds = cocoGt.getAnnIds(imgId, VOC_CAT_IDS)
                for ann in cocoGt.loadAnns(annIds):
                    mask = cocoGt.annToMask(ann)
                    output_tensor[COCO_TO_VOC[ann['category_id']]] |= mask
                    
                # Set everything not labeled to be background
                output_tensor[0] = 1 - np.max(output_tensor, axis=0)
                self.data.append((input_tensor, output_tensor))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def getImgIdsUnion(self, gt, catIds):
        """
        Returns all the images that have *any* of the categories in `catIds`,
        unlike the built-in `gt.getImgIds` which returns all the images containing
        *all* of the categories in `catIds`.
        """
        imgIds = set()
        for catId in catIds:
            imgIds |= set(gt.catToImgs[catId])
        return list(imgIds)
        
    def load_image(self, img_path):
        input_image = Image.open(img_path).convert('RGB')
        input_tensor = preprocess(input_image)
        input_tensor = input_tensor.detach().cpu().numpy()
        return input_tensor


In [6]:
def iou(model_tensor, target_tensor):
    # Don't include the background when summing
    model_tensor = model_tensor[:, 1:, :, :]
    target_tensor = target_tensor[:, 1:, :, :]
    
    intersection = np.sum(np.logical_and(model_tensor, target_tensor))
    union = np.sum(np.logical_or(model_tensor, target_tensor))
    
    if union == 0:
        # Can only happen if nothing was there and nothing was predicted,
        # which is a perfect score
        return 1
    else:
        return intersection / union

def evaluate(model, dataloader):    
    totalIoU = 0
    sess = onnxruntime.InferenceSession(model.SerializeToString(),
                                        None,
                                        providers=onnxruntime.get_available_providers())
    idx = 1
    for input_tensor, target_tensor in dataloader:
        input_tensor = input_tensor[np.newaxis, ...]
        target_tensor = target_tensor[np.newaxis, ...]
        model_tensor = sess.run(["out"], {"input": input_tensor})[0]
        
        batch_size, nclasses, height, width = model_tensor.shape
        raw_labels = np.argmax(model_tensor, axis=1).astype(np.uint8)
        
        output_tensor = np.zeros((nclasses, batch_size, height, width), dtype=np.uint8)
        for c in range(nclasses):
            output_tensor[c][raw_labels==c] = 1

        output_tensor = np.transpose(output_tensor, [1, 0, 2, 3])          
        totalIoU += iou(output_tensor, target_tensor)    
        idx += 1
    return totalIoU / idx


In [7]:
def eval(model):
  return evaluate(model, ds)

In [8]:
model = onnx.load("models/fcn-resnet50-12.onnx")
ds = Dataset()

In [9]:
from neural_compressor.experimental import Quantization, common
from neural_compressor import options
options.onnxrt.graph_optimization.level = 'ENABLE_BASIC'

quantize = Quantization("fcn_rn50.yaml")
quantize.model = common.Model(model)
quantize.calib_dataloader = common.DataLoader(ds)
quantize.eval_func = eval
q_model = quantize()
q_model.save("models/quantized/fcn-resnet50-int8.onnx")

2022-12-16 23:01:58 [INFO] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-12-16 23:01:58 [INFO] NumExpr defaulting to 8 threads.
2022-12-16 23:01:59.932019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 23:02:00.077560: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-16 23:02:00.526678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nana/anaconda3/envs/tf

In [11]:
from neural_compressor.experimental import Benchmark, common

model = onnx.load("models/fcn-resnet50-12.onnx")

evaluator = Benchmark("fcn_rn50.yaml")
evaluator.model = common.Model(model)
evaluator.b_dataloader = common.DataLoader(ds)
evaluator("performance")

2022-12-16 23:05:56 [INFO] Start to run Benchmark.
2022-12-16 23:06:06 [INFO] 
performance mode benchmark result:
2022-12-16 23:06:06 [INFO] Batch size = 1
2022-12-16 23:06:06 [INFO] Latency: 353.151 ms
2022-12-16 23:06:06 [INFO] Throughput: 2.832 images/sec


In [12]:
model = onnx.load("models/quantized/fcn-resnet50-int8.onnx")

evaluator = Benchmark("fcn_rn50.yaml")
evaluator.model = common.Model(model)
evaluator.b_dataloader = common.DataLoader(ds)
evaluator("performance")

2022-12-16 23:06:45 [INFO] Start to run Benchmark.
2022-12-16 23:06:53 [INFO] 
performance mode benchmark result:
2022-12-16 23:06:53 [INFO] Batch size = 1
2022-12-16 23:06:53 [INFO] Latency: 289.919 ms
2022-12-16 23:06:53 [INFO] Throughput: 3.449 images/sec
